In [1]:
import pandas as pd
import numpy as np
import requests
import os
import glob
from bs4 import BeautifulSoup

# Get Variable Definitions

In [2]:
def get_variable_description(url):

    # get the html page with the variable codebook for all xpt files
    # use this to rename the columns to descriptive cols
    page = requests.get(url)

    # beautiful soup is good for parsing html
    soup = BeautifulSoup(page.content, 'html.parser')

    # this is the id for the table on the variable codebook bage
    tbl = soup.find("table",{"id":"GridView1"})

    # get the table and load into a df
    df_var_mapping = pd.read_html(str(tbl))[0]
    return df_var_mapping


questionnaire_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Questionnaire&Cycle=2017-2020'
)

demo_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics&Cycle=2017-2020'
)

exam_var_desc = get_variable_description(
    url = 'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Examination&Cycle=2017-2020'
)

df_var_mapping = pd.concat([questionnaire_var_desc,demo_var_desc,exam_var_desc])
#df_var_mapping

df_var_mapping.to_csv("../data/cdc_nhanes_var_mapping.csv")

# Read XPT Data

In [3]:
def read_xpt_files(folder_path):
    """Input folder path to read multiple xpt files in folder
       Returns dictionary with key: file name, 
                               value: df"""
    df_dict = {}
    for file_name in os.listdir(folder_path):
        if 'XPT' in os.path.splitext(file_name)[1]:
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_sas(file_path, format='xport')
            name = os.path.splitext(file_name)[0]
            if df.empty:
                raise Exception (f'Empty dataframe from file: {name}')
            df_dict[name] = df
        else:
            print(f'not loading file {file_name}')
    return df_dict



def full_outer_join(dataframes):
    joined_df = None
    for df in dataframes.values():
        if joined_df is None:
            joined_df = df
        else:
            joined_df = pd.merge(joined_df, df, on='SEQN', how='outer')
    return joined_df

## 2017 - March 2020 Data

In [4]:
# all data except for prescription medicine

folder_path = '../data/xpt_data/2017_march2020'
dataframes_dict = read_xpt_files(folder_path)
df_cdc_joined_pre = full_outer_join(dataframes_dict)
# df_cdc_joined_pre

not loading file prescription_med
not loading file .ipynb_checkpoints


In [5]:
# prescription medicine survey - need to dedup

folder_path = '../data/xpt_data/2017_march2020/prescription_med'
dataframes_dict = read_xpt_files(folder_path)
prescript_med = full_outer_join(dataframes_dict)

prescript_med['RXDRSD1']=prescript_med['RXDRSD1'].apply(lambda x: x.decode())
prescript_med['RXDRSD2']=prescript_med['RXDRSD2'].apply(lambda x: x.decode())
prescript_med['RXDRSD3']=prescript_med['RXDRSD3'].apply(lambda x: x.decode())

conditions = [
    (prescript_med["RXDRSD1"] == "Major depressive disorder, single episode, unspecified"),
    (prescript_med["RXDRSD1"] == "Major depressive disorder, recurrent, unspecified"),
    (prescript_med["RXDRSD2"] == "Major depressive disorder, single episode, unspecified"),
    (prescript_med["RXDRSD2"] == "Major depressive disorder, recurrent, unspecified"),
    (prescript_med["RXDRSD3"] == "Major depressive disorder, single episode, unspecified"),
    (prescript_med["RXDRSD3"] == "Major depressive disorder, recurrent, unspecified")
    ]

values = [1,1,1,1,1,1]
prescript_med['MDD'] = np.select(conditions, values)


# Sort by SEQN and MDD for any instances where the responder has MDD == 1, it will precede any rows where MDD == 0
# Only keep first row of each responder to retain rows where MDD == 1 if possible and MDD == 0 where not
prescript_med.sort_values(by=["SEQN"], axis = 0, ascending =[True], inplace=True)
prescript_med.sort_values(by=["MDD"], axis = 0, ascending =[False], inplace=True)
pm_dropped = prescript_med.drop_duplicates(subset = ['SEQN'], keep = 'first', inplace = False) 
pm_dropped = pm_dropped[['SEQN','MDD']]
# pm_dropped

In [6]:
# merge into 1 

df_cdc_joined = pd.merge(df_cdc_joined_pre, pm_dropped, on='SEQN', how='left')
df_cdc_joined

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,SMQ078,SMD641,SMD650,SMD100FL,SMD100MN,SMQ670,SMQ621,SMD630,SMAQUEX2,MDD
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0,0
2,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,30.0,15.0,1.0,1.0,1.0,NaN,NaN,1.0,0
3,109274.0,1.0,4.000000e+00,2.0,5.000000e+00,7.0,5.397605e-79,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
4,109282.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,121170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15556,121709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15557,122778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15558,122879.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# Sanitize Data

In [7]:
# convert bytes and strip whitespace
df_cdc_joined_clean = df_cdc_joined.apply(lambda x: x.str.decode('utf-8').str.strip() if x.dtype == "object" else x)
# replace empty strings with nan
df_cdc_joined_clean = df_cdc_joined_clean.replace('', np.nan)
df_cdc_joined_clean

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,SMQ078,SMD641,SMD650,SMD100FL,SMD100MN,SMQ670,SMQ621,SMD630,SMAQUEX2,MDD
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,2.0,30.0,20.0,NaN,NaN,1.0,NaN,NaN,1.0,0
2,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,30.0,15.0,1.0,1.0,1.0,NaN,NaN,1.0,0
3,109274.0,1.0,4.000000e+00,2.0,5.000000e+00,7.0,5.397605e-79,NaN,2.0,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
4,109282.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,121170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15556,121709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15557,122778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
15558,122879.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## Clean up ordinal variables

In [8]:
def clean_ordinal_col(x, column, max_number):
    
    col_value = round(x[column])
    acceptable_values = list(range(max_number+1))
    
    if col_value in acceptable_values:
        new_val = col_value
    else:
        new_val = None

    return new_val

ordinal_column_key = {
    'DPQ010':('little_interest_in_doing_things',3),
    'DPQ020':('feeling_down_depressed_hopeless',3),
    'DPQ030':('trouble_falling_or_staying_asleep',3),
    'DPQ040':('feeling_tired_or_having_little_energy',3),
    'DPQ050':('poor_appetitie_or_overeating',3),
    'DPQ060':('feeling_bad_about_yourself',3),
    'DPQ070':('trouble_concentrating',3),
    'DPQ080':('moving_or_speaking_to_slowly_or_fast',3),
    'DPQ090':('thoughts_you_would_be_better_off_dead',3),
    'DPQ100':('difficult_doing_daytoday_tasks',3),
    'ALQ121':('alcoholic_drinks_past_12mo',10),
    'DBQ700':('how_healthy_is_your_diet',5),
    'DID250':('count_days_seen_doctor_12mo',40),
    'PAQ670':('count_days_moderate_recreational_activity',7),   
    'PAD675':('count_minutes_moderate_recreational_activity',600),
    'PAD680':('count_minutes_moderate_sedentary_activity',1320),
    'PAQ706':('count_days_physical_activity_youth',7),   
    'WHQ225':('count_lost_10plus_pounds',4),
    'WHQ520':('count_tried_to_lose_weight_youth',3),
    'FSDHH':('food_security_level_household',4),   
    'FSDAD':('food_security_level_adult',4),
    'HUQ010':('general_health_condition',5),
    'HUD062':('duration_last_healthcare_visit',4),      
    'INDFMMPC':('monthly_poverty_index_category',3),
    'OCQ180':('count_hours_worked_last_week',80),
    'RIDAGEYR':('age_in_years',80),   
    'DMDEDUC2':('education_level',5)        
}

In [9]:
# for each ordinal variable create a new column that removes all the "missing response" or "don't know" so
# it can be treated as a range.

# For example, for column "DPQ010" values 7 = "Refused" & values 9 = "Don't know".
# The max_number for that column is equal to 3 so we set all values > 3 as null.

for key in ordinal_column_key.keys():
    
    col_name = key
    new_col_name = ordinal_column_key[key][0]
    max_num = ordinal_column_key[key][1]
    
    df_cdc_joined_clean[new_col_name] = df_cdc_joined_clean.apply(
        lambda x: clean_ordinal_col(x = x, column = col_name, max_number = max_num), axis = 1
    )

## Clean up category variables

In [10]:
def clean_category_col(x, column):
    
    col_value = x[column]
    
    if col_value == 1:
        new_val = 1
    elif col_value == 2:
        new_val = 0
    else:
        new_val = None

    return new_val

category_column_key = {
    'HIQ011':'has_health_insurance',
    'HIQ210':'has_health_insurance_gap',
    'SMQ681':'has_smoked_tabacco_last_5days',
    'RIAGENDR':'is_male',
    'DMDBORN4':'is_usa_born',
    'DIQ010':'has_diabetes',
    'MCQ080':'has_overweight_diagnosis',    
    'WHQ070':'has_tried_to_lose_weight_12mo',
    'RHQ131':'has_been_pregnant'
}

In [11]:
for key in category_column_key.keys():
    
    col_name = key
    new_col_name = category_column_key[key]
    
    df_cdc_joined_clean[new_col_name] = df_cdc_joined_clean.apply(
        lambda x: clean_category_col(x = x, column = col_name), axis = 1
    )

## one-off columns

In [12]:
df_cdc_joined_clean['monthly_poverty_index'] = df_cdc_joined_clean['INDFMMPI']

In [13]:
df_cdc_joined_clean

,SEQN,ALQ111,ALQ121,ALQ130,ALQ142,ALQ270,ALQ280,ALQ290,ALQ151,ALQ170,...,has_health_insurance,has_health_insurance_gap,has_smoked_tabacco_last_5days,is_male,is_usa_born,has_diabetes,has_overweight_diagnosis,has_tried_to_lose_weight_12mo,has_been_pregnant,monthly_poverty_index
0,109266.0,1.0,1.000000e+01,1.0,5.397605e-79,NaN,NaN,NaN,2.0,5.397605e-79,...,1.0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,5.00
1,109271.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.0,0.0,1.0,1,1.0,0.0,0.0,0.0,NaN,1.20
2,109273.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,1.0,1.0,1,1.0,0.0,1.0,0.0,NaN,0.53
3,109274.0,1.0,4.000000e+00,2.0,5.000000e+00,7.0,5.397605e-79,NaN,2.0,5.397605e-79,...,1.0,0.0,0.0,1,1.0,1.0,1.0,NaN,NaN,1.20
4,109282.0,1.0,5.397605e-79,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,1.0,0.0,0.0,1,1.0,0.0,0.0,0.0,NaN,3.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15555,121170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,1,1.0,NaN,NaN,NaN,NaN,0.49
15556,121709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,0,1.0,NaN,NaN,NaN,NaN,2.63
15557,122778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,0,1.0,NaN,NaN,NaN,NaN,0.82
15558,122879.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,NaN,0,1.0,NaN,NaN,NaN,NaN,2.13


In [16]:
df_cdc_joined_clean.to_csv("../data/cdc_nhanes_survey_responses_clean.csv")

# clean up additional columns with high correlation with outcome variable

In [ ]:
df_clean = df_cdc_joined_clean
#filter for women who have been pregnant in calculating correlation but keep all responses
preg = df_clean[df_clean['has_been_pregnant']==1]
df=preg

#convert to floats to enable correlation calculations
for col in df:
    df[col] = df[col].astype(float)


df_corr=df.corr(method='pearson')
corr_values=(df_corr['MDD']).sort_values(ascending=False)
df_corr_values=corr_values.to_frame()

#Identify features with higher correlation with outcome variable
filtered_corr = df_corr_values[df_corr_values['MDD'] > 0.1]
high_corr_feat = filtered_corr.index.tolist()


"""
Cleaned features:
DPQ020 = feeling_down_depressed_hopeless
DPQ040 = feeling_tired_or_having_little_energy
DPQ060 = feeling_bad_about_yourself
DPQ070 = trouble_concentrating
DPQ100 = difficult_doing_daytoday_tasks
DPQ050 = poor_appetitie_or_overeating
DPQ010 = little_interest_in_doing_things
DPQ030 = trouble_falling_or_staying_asleep
HUQ010 = general_health_condition
DPQ080 = moving_or_speaking_to_slowly_or_fast
WHQ225 = count_lost_10plus_pounds
DBQ700 = how_healthy_is_your_diet
DPQ090 = thoughts_you_would_be_better_off_dead
DMDBORN4 = is_usa_born
FSDAD = food_security_level_adult
MCQ080 = has_overweight_diagnosis
FSDHH = food_security_level_household
RIDAGEYR = age_in_years
DID250 = count_days_seen_doctor_12mo

Feautures to clean:
done - MCD180D = age_with_angina_pectoris (convert 77777 and 99999 to NaN)
done - RHQ020 = age_range_first_menstrual_period (convert 7 and 9 to NaN)
done - MCQ170L = have_liver_condition (convert 7 and 9 to NaN)
BMIHT = BMI_standing_height_comment (drop column)
done - MCD180L = age_liver_condition (convert 77777 and 99999 to NaN)
done - OCD150 = type_of_work_done_last_week (convert 7 and 9 to NaN)
done - WHQ060 = weight_change_intentional (convert 7 and 9 to NaN)
done - SMQ830 = days_nicotine_substitute_used (convert 7 and 9 to NaN)
done - BMXWAIST = waist_circumference (no cleaning needed)
done - CDQ006 = pain_relief_from_cardio_recoverytime (convert 7 and 9 to NaN)
done - HUQ051 = annual_healthcare_visit_count (convert 77 and 99 to NaN)

"""

#keep all original columns and create new sanitized columns with more descriptive names 

def sanitize_5(value):
    if value == 77777 or value == 99999:
        return np.nan
    else:
        return value
    
def sanitize_3(value):
    if value == 777 or value == 999:
        return np.nan
    else:
        return value
    
def sanitize_2(value):
    if value == 77 or value == 99:
        return np.nan
    else:
        return value
    
def sanitize_1(value):
    if value == 7 or value == 9:
        return np.nan
    else:
        return value
    
def convert_to_int(df):
    converted_columns = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = (df[col].fillna(0)
                      .astype(int)
                      .astype(object)
                      .where(df[col].notnull()))
            converted_columns.append(col)
    if converted_columns == []:
        print('No columns were converted')
    else:
        print('Converted columns ', converted_columns, ' to integer type')

#convert all columns to int
convert_to_int(df_clean)

df_clean['age_with_angina_pectoris'] = df_clean['MCD180D'].map(sanitize_5)
df_clean['age_liver_condition'] = df_clean['MCD180L'].map(sanitize_5)
df_clean['age_range_first_menstrual_period'] = df_clean['RHQ020'].map(sanitize_1)
df_clean['annual_healthcare_visit_count'] = df_clean['HUQ051'].map(sanitize_2)
df_clean['have_liver_condition'] = df_clean['MCQ170L'].map(sanitize_1)
df_clean['type_of_work_done_last_week'] = df_clean['OCD150'].map(sanitize_1)
df_clean['weight_change_intentional'] = df_clean['WHQ060'].map(sanitize_1)
df_clean['days_nicotine_substitute_used'] = df_clean['SMQ830'].map(sanitize_1)
df_clean['pain_relief_from_cardio_recoverytime'] = df_clean['CDQ006'].map(sanitize_1)
df_clean = df_clean.drop('BMIHT', axis=1)

df_clean.to_csv('df_cdc_clean_v2.csv', index=False)